In [1]:
import requests
import uuid
import json
import time
import os
import random

# ПОЛУЧАЕМ ДАННЫЕ О СПИСКЕ КОЛЛЕКЦИЙ

In [9]:
id = str(uuid.uuid4())
message = { "id": id, "mqtt_key": "234", "user": "2321", "type": "courses", "data_key": "","action": "get_data" }
resp = requests.post("http://localhost:57888/api/add_message", json=message)
resp

<Response [200]>

In [10]:
message = { "id": id }
resp = requests.post("http://localhost:57888/api/get_message_result", json=message)
resp.json()

{'message': {'id': '9f8ef80d-2a81-4628-b441-351f15281420',
  'mqtt_key': '234',
  'user': '2321',
  'type': 'courses',
  'data_key': '',
  'action': 'get_data',
  'timestamp': 1671697922},
 'result': {'courses': ['test']}}

# ПОЛУЧАЕМ ДАННЫЕ О СТРУКТУРЕ КОЛЛЕКЦИИ (АСИНХРОННЫЙ СПОСОБ)

In [11]:
id = str(uuid.uuid4())
message = { "id": id, "mqtt_key": "234", "user": "2321", "type": "problems", "data_key": "test", "action": "get_data" }
resp = requests.post("http://localhost:57888/api/add_message", json=message)
resp

<Response [200]>

In [12]:
message = { "id": id }
resp = requests.post(
    "http://localhost:57888/api/get_message_result", json=message)
resp


<Response [200]>

In [13]:
resp.json()

{'message': {'id': '9ca156d9-10d0-4dce-8f44-649a549ba0b7',
  'mqtt_key': '234',
  'user': '2321',
  'type': 'problems',
  'data_key': 'test',
  'action': 'get_data',
  'timestamp': 1671697929},
 'result': {'problems': [{'1': ['1'],
    'rating': 1,
    'task': '1. Напишите программу для знакомства с человеком и приветствия его (пользователь вводит Имя и Фамилию в двух отдельных строках, программа выводит в новой строке приветствие, например, при вводе имени «Иван» и фамилии «Петров» программа выводит : «Здравствуйте, Иван Петров !»)'},
   {'2': ['1'],
    'rating': 3,
    'task': '2. Напишите программу, которая выбирает максимальное и минимальное из пяти введенных чисел.'},
   {'3': ['1'],
    'rating': 2,
    'task': '3. Напишите программу, которая находит среднее из пяти введенных чисел.'},
   {'4': ['1'],
    'rating': 2,
    'task': '4. Найти периметр и площадь прямоугольного треугольника, если даны длины его катетов a и b. Значения а и b вводятся с клавиатуры.'},
   {'5': ['1'],
 

# ПОЛУЧАЕМ ДАННЫЕ О СТРУКТУРЕ КОЛЛЕКЦИИ (СИНХРОННЫЙ СПОСОБ)

In [26]:
id = str(uuid.uuid4())
message = {"id": id, "mqtt_key": "234", "user": "2321",
           "type": "problems", "data_key": "test", "action": "get_data"}
resp = requests.post(
    "http://localhost:57888/api/get_courses_data", json=message)
resp

<Response [200]>

In [27]:
resp.json()

{'data': {'problems': [{'1': ['1'],
    'rating': 1,
    'task': '1. Напишите программу для знакомства с человеком и приветствия его (пользователь вводит Имя и Фамилию в двух отдельных строках, программа выводит в новой строке приветствие, например, при вводе имени «Иван» и фамилии «Петров» программа выводит : «Здравствуйте, Иван Петров !»)'},
   {'2': ['1'],
    'rating': 3,
    'task': '2. Напишите программу, которая выбирает максимальное и минимальное из пяти введенных чисел.'},
   {'3': ['1'],
    'rating': 2,
    'task': '3. Напишите программу, которая находит среднее из пяти введенных чисел.'},
   {'4': ['1'],
    'rating': 2,
    'task': '4. Найти периметр и площадь прямоугольного треугольника, если даны длины его катетов a и b. Значения а и b вводятся с клавиатуры.'},
   {'5': ['1'],
    'rating': 3,
    'task': '5. Дана длина ребра куба. Найти площадь грани, площадь полной поверхности и объем этого куба. Длина ребра куба вводится с клавиатуры'},
   {'6': ['1'],
    'rating': 3

# УДАЛЕНИЕ КОЛЛЕКЦИИ

In [7]:
id = str(uuid.uuid4())
message = {"id": id, "mqtt_key": "234", "user": "2321",
           "type": "course", "data_key": "test1", "action": "clear_data"}
resp = requests.post(
    "http://localhost:57888/api/clear_data", json=message)
resp

<Response [200]>

In [5]:
resp.json()

{'data': {'test': 'Droped!'}}

# СОСТАВЛЕНИЕ ФАЙЛА С ЗАДАЧАМИ

In [6]:
path = "/Users/user/Temp/vstu-pycourse-tests"
tests = []
for number in range(1, 90):
    fn = f'{path}/tests/{number}/result.json'
    fn1 = f'{path}/tasks/{number}.py'
    if os.path.exists(fn):
        with open(fn, encoding='utf-8') as f:
            var = json.load(f)
        with open(fn1, encoding='utf-8') as f:
            txt = f.read()
            ind1 = txt.find("'''")
            ind2 = txt.find("'''", 10)
            txt = txt[ind1+3:ind2].replace("\n", " ")
        test = {"problem": number, "rating": random.choice([1,2,3]), "task": txt, "type": "equal", "variants": {"1": var}}
        tests.append(test)
with open(f"{path}/test_problems.json", "w", encoding='utf-8') as f:
    json.dump(tests, f, ensure_ascii=False)

# ДОБАВЛЕНИЕ ЗАДАЧ ИЗ ФАЙЛА

In [6]:
course = "test"
#addr = "http://cluster.vstu.ru:57888"
addr = "http://localhost:57888"
with open("test_problems.json", encoding='utf-8') as f:
    data = json.load(f)
for p in data:
    for v in p["variants"]:
        print(f'Problem {p["problem"]}, variant {v}')
        id = str(uuid.uuid4())
        message = {
            "id": id, 
            "mqtt_key": 123,
            "user": "admin", 
            "type": p["type"],
            "rating": p["rating"],
            "course": course, 
            "problem": p["problem"], 
            "variant": v,
            "action": "add_problem",
            "task": p.get("task", ""),
            "tests": p["variants"][v]
        }
        r = requests.post(f"{addr}/api/add_message", json=message)
        print(f"{r}")
        message = { "id": id }
        while True:
            time.sleep(1)
            resp = requests.post(f"{addr}/api/get_message_result", json=message)
            if resp.status_code == 200 and 'message' in resp.text:
                break
                print(f"{resp.json()}")


Problem 1, variant 1
<Response [200]>
Problem 2, variant 1
<Response [200]>
Problem 3, variant 1
<Response [200]>
Problem 4, variant 1
<Response [200]>
Problem 5, variant 1
<Response [200]>
Problem 6, variant 1
<Response [200]>
Problem 7, variant 1
<Response [200]>
Problem 8, variant 1
<Response [200]>
Problem 9, variant 1
<Response [200]>
Problem 10, variant 1
<Response [200]>
Problem 11, variant 1
<Response [200]>
Problem 12, variant 1
<Response [200]>
Problem 13, variant 1
<Response [200]>
Problem 14, variant 1
<Response [200]>
Problem 15, variant 1
<Response [200]>
Problem 16, variant 1
<Response [200]>
Problem 17, variant 1
<Response [200]>
Problem 18, variant 1
<Response [200]>
Problem 19, variant 1
<Response [200]>
Problem 20, variant 1
<Response [200]>
Problem 21, variant 1
<Response [200]>
Problem 22, variant 1
<Response [200]>
Problem 23, variant 1
<Response [200]>
Problem 24, variant 1
<Response [200]>
Problem 25, variant 1
<Response [200]>
Problem 26, variant 1
<Response [2

# ПОЛУЧАЕМ ДАМП БАЗЫ

In [64]:
message = { "date": "2022-12-18", "processor_name": "equal_processor", "admin_key": "e86e915a-249f-4fdc-a48a-7c3f9a0b36ff" }
resp = requests.post(
    "http://cluster.vstu.ru:57888/api/get_base_dump", json=message)
resp

<Response [200]>

In [65]:
resp.json()["data"][-1]

{'message': {'id': '7cf6663d-f204-48f6-ad46-be596b00ad50',
  'mqtt_key': '123',
  'user': 'qwe',
  'language': 'python3',
  'course': 'test',
  'action': 'test_problem',
  'problem': '1',
  'variant': '1',
  'code': "'''1. Напишите программу для знакомства с человеком и приветствия его\n(пользователь вводит Имя и Фамилию в двух отдельных строках, программа\nвыводит в новой строке приветствие, например, при вводе имени «Иван» и\nфамилии «Петров» программа выводит : «Здравствуйте, Иван Петров !»)'''\n\nprint(f'Здравствуйте, {input()} {input()} !')\n",
  'timestamp': 1671372911},
 'result': {'1': {'score': 10, 'test_out': 'Здравствуйте, Иван Дремин !'},
  '3': {'score': 10, 'test_out': 'Здравствуйте, Madi Ansir !'},
  '2': {'score': 10, 'test_out': 'Здравствуйте, Илья Артёмов !'},
  'success_count': 3,
  'res_score': 30,
  'max_res_score': 30,
  'timestamp': 1671372912}}

In [63]:
resp.json()

{'error': 'Need admin_key for process request!'}

# ДОБАВЛЯЕМ/ИЗМЕНЯЕМ ИНФОРМАЦИЮ ПОЛЬЗОВАТЕЛЯ

In [3]:
message = { "user_name": "123456", "data": {"var": [1,2,3]} }
resp = requests.post(
    "http://localhost:57888/api/add_user_info", json=message)
resp

<Response [200]>

In [35]:
resp.json()

{'data': 'sucess'}

# ПОЛУЧАЕМ ИНФОРМАЦИЮ ПОЛЬЗОВАТЕЛЯ

In [17]:
message = { "user_name": "1234" }
resp = requests.post(
    "http://localhost:57888/api/get_user_info", json=message)
resp

<Response [200]>

In [18]:
resp.json()

{'data': {'user_name': '1234',
  'data': {'test': [{'2': ['1'],
     'rating': 3,
     'task': '2. Напишите программу, которая выбирает максимальное и минимальное из пяти введенных чисел.'},
    {'35': ['1'],
     'rating': 3,
     'task': '35.  Напишите программу, которая вводит с клавиатуры натуральное число  N и определяет его факториал, т.е. произведение натуральных чисел от 1 до N.  Что будет, если ввести большое значение N (например, N = 20)?'},
    {'43': ['1'],
     'rating': 1,
     'task': '43.  Напишите программу, которая вводит с клавиатуры числа до тех пор,  пока не будет введено число 0. В конце работы программы на экран выводится  максимальное и минимальное из введенных чисел (не считая 0).'}]}}}